<a href="https://colab.research.google.com/github/Nachostaco/HabitablePlanetRegression/blob/main/ExoplanetHabitabilityClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import regex as re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/My Drive/ExHaCl/hwc.csv') as f:
  data = pd.read_csv(f, usecols=lambda x: x not in ["S_CONSTELLATION_ABR",	'S_CONSTELLATION_ENG', 'S_NAME_HD', 'S_NAME_HIP'])
data.head()

,P_NAME,P_DETECTION,P_DISCOVERY_FACILITY,P_YEAR,P_UPDATE,P_MASS,P_MASS_ERROR_MIN,P_MASS_ERROR_MAX,P_MASS_LIMIT,P_MASS_ORIGIN,P_RADIUS,P_RADIUS_ERROR_MIN,P_RADIUS_ERROR_MAX,P_RADIUS_LIMIT,P_PERIOD,P_PERIOD_ERROR_MIN,P_PERIOD_ERROR_MAX,P_PERIOD_LIMIT,P_SEMI_MAJOR_AXIS,P_SEMI_MAJOR_AXIS_ERROR_MIN,P_SEMI_MAJOR_AXIS_ERROR_MAX,P_SEMI_MAJOR_AXIS_LIMIT,P_ECCENTRICITY,P_ECCENTRICITY_ERROR_MIN,P_ECCENTRICITY_ERROR_MAX,P_ECCENTRICITY_LIMIT,P_INCLINATION,P_INCLINATION_ERROR_MIN,P_INCLINATION_ERROR_MAX,P_INCLINATION_LIMIT,P_OMEGA,P_OMEGA_ERROR_MIN,P_OMEGA_ERROR_MAX,P_OMEGA_LIMIT,S_NAME,S_TYPE,S_RA,S_DEC,S_RA_STR,S_DEC_STR,...,P_ESCAPE,P_POTENTIAL,P_GRAVITY,P_DENSITY,P_HILL_SPHERE,P_DISTANCE,P_PERIASTRON,P_APASTRON,P_DISTANCE_EFF,P_FLUX,P_FLUX_MIN,P_FLUX_MAX,P_TEMP_EQUIL,P_TEMP_EQUIL_MIN,P_TEMP_EQUIL_MAX,P_TEMP_SURF,P_TEMP_SURF_MIN,P_TEMP_SURF_MAX,P_TYPE,S_TYPE_TEMP,S_RA_TXT,S_DEC_TXT,S_LUMINOSITY,S_HZ_OPT_MIN,S_HZ_OPT_MAX,S_HZ_CON_MIN,S_HZ_CON_MAX,S_HZ_CON0_MIN,S_HZ_CON0_MAX,S_HZ_CON1_MIN,S_HZ_CON1_MAX,S_SNOW_LINE,S_ABIO_ZONE,S_TIDAL_LOCK,P_HABZONE_OPT,P_HABZONE_CON,P_TYPE_TEMP,P_HABITABLE,P_ESI,S_CONSTELLATION
0,OGLE-2016-BLG-1227L b,Microlensing,OGLE,2020,2020-04-02,250.00,-120.0,413.0,0,Mass,13.90,NaN,NaN,0,NaN,NaN,NaN,0,3.4000,-1.0000,2.1000,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,OGLE-2016-BLG-1227L,NaN,265.597130,-33.759778,17h42m23.31s,-33d45m35.2s,...,4.240945,17.985612,1.293929,0.093088,0.461577,3.4000,3.4000,3.4000,3.4000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jovian,NaN,17 42 23.31,-33 45 35.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.214133,0,0,NaN,0,NaN,Scorpius
1,Kepler-276 c,Transit,Kepler,2013,2018-09-25,16.60,-3.6,4.4,0,Mass,2.90,-0.28,1.27,0,31.884000,NaN,NaN,0,0.1994,NaN,NaN,0,0.0,NaN,NaN,0,86.92,NaN,NaN,0,NaN,NaN,NaN,0,Kepler-276,NaN,293.568200,39.036312,19h34m16.37s,+39d02m10.72s,...,2.392517,5.724138,1.973841,0.680635,0.004929,0.1994,0.1994,0.1994,0.1994,20.490365,20.490365,20.490365,541.98483,541.98483,541.98483,NaN,612.08670,612.08670,Neptunian,G,19 34 16.37,+39 02 10.72,0.814704,0.677337,1.595742,0.857931,1.512908,0.907212,1.512908,0.828167,1.512908,2.437046,2.097783,0.316980,0,0,Hot,0,0.272032,Cygnus
2,Kepler-829 b,Transit,Kepler,2016,2019-04-16,5.10,NaN,NaN,0,M-R relationship,2.11,-0.18,0.44,0,6.883376,-0.000020,0.000020,0,0.0678,NaN,NaN,0,0.0,NaN,NaN,0,89.82,NaN,NaN,0,NaN,NaN,NaN,0,Kepler-829,NaN,282.332830,42.463813,18h49m19.88s,+42d27m49.73s,...,1.554690,2.417062,1.145527,0.542904,0.001175,0.0678,0.0678,0.0678,0.0678,238.528680,238.528680,238.528680,1001.11710,1001.11710,1001.11710,1130.60440,1130.60440,1130.60440,Superterran,G,18 49 19.88,+42 27 49.73,1.096478,0.789603,1.864342,1.000132,1.767566,1.057660,1.767566,0.965447,1.767566,2.827247,1.756317,0.459559,0,0,Hot,0,0.254763,Lyra
3,K2-283 b,Transit,K2,2018,2019-09-05,12.20,NaN,NaN,0,M-R relationship,3.52,-0.16,0.16,0,1.921036,-0.000053,0.000051,0,0.0291,-0.0003,0.0003,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,K2-283,NaN,13.194368,9.692918,00h52m46.65s,+09d41m34.51s,...,1.861695,3.465909,0.984633,0.279725,0.000697,0.0291,0.0291,0.0291,0.0291,353.357260,353.357260,353.357260,1104.46820,1104.46820,1104.46820,NaN,1247.32330,1247.32330,Neptunian,K,00 52 46.65,+09 41 34.51,0.299226,0.427216,1.030473,0.541131,0.976981,0.572630,0.976981,0.522400,0.976981,1.476943,0.568374,0.443760,0,0,Hot,0,0.193906,Pisces
4,Kepler-477 b,Transit,Kepler,2016,2019-04-16,4.94,NaN,NaN,0,M-R relationship,2.07,-0.12,0.18,0,11.119907,-0.000012,0.000012,0,0.0911,NaN,NaN,0,0.0,NaN,NaN,0,87.75,NaN,NaN,0,NaN,NaN,NaN,0,Kepler-477,NaN,288.067450,42.355305,19h12m16.19s,+42d21m19.10s,...,1.544822,2.386473,1.152886,0.556950,0.001626,0.0911,0.0911,0.0911,0.0911,51.163853,51.163853,51.163853,681.30334,681.30334,681.30334,769.42507,769.42507,769.42507,Superterran,K,19 12 16.19,+42 21 19.10,0.424620,0.504264,1.208168,0.638721,1.145452,0.675769,1.145452,0.616604,1.145452,1.759397,0.768502,0.386150,0,0,Hot,0,0.276721,Lyra


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5599 entries, 0 to 5598
Columns: 114 entries, P_NAME to S_CONSTELLATION
dtypes: float64(81), int64(18), object(15)
memory usage: 4.9+ MB


In [ ]:
columns_with_nans: list = data.columns[data.isnull().any()].to_list()
data[columns_with_nans].isna().sum()

,0
P_MASS,7
P_MASS_ERROR_MIN,3093
P_MASS_ERROR_MAX,3093
P_RADIUS,7
P_RADIUS_ERROR_MIN,1752
...,...
S_SNOW_LINE,233
S_ABIO_ZONE,223
S_TIDAL_LOCK,4
P_TYPE_TEMP,234


Error and limit columns are not useful in a habitability classification, so I just drop them

In [ ]:
error_columns: list = data.columns[data.columns.str.contains(r'(?:ERROR|LIMIT)')].to_list()
data.drop(columns=error_columns, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5599 entries, 0 to 5598
Data columns (total 68 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   P_NAME                5599 non-null   object 
 1   P_DETECTION           5599 non-null   object 
 2   P_DISCOVERY_FACILITY  5599 non-null   object 
 3   P_YEAR                5599 non-null   int64  
 4   P_UPDATE              5599 non-null   object 
 5   P_MASS                5592 non-null   float64
 6   P_MASS_ORIGIN         5599 non-null   object 
 7   P_RADIUS              5592 non-null   float64
 8   P_PERIOD              5350 non-null   float64
 9   P_SEMI_MAJOR_AXIS     5595 non-null   float64
 10  P_ECCENTRICITY        4822 non-null   float64
 11  P_INCLINATION         4288 non-null   float64
 12  P_OMEGA               1659 non-null   float64
 13  S_NAME                5599 non-null   object 
 14  S_TYPE                2021 non-null   object 
 15  S_RA                 

In [ ]:
data[data.select_dtypes(include='object').columns.to_list()]

,P_NAME,P_DETECTION,P_DISCOVERY_FACILITY,P_UPDATE,P_MASS_ORIGIN,S_NAME,S_TYPE,S_RA_STR,S_DEC_STR,P_TYPE,S_TYPE_TEMP,S_RA_TXT,S_DEC_TXT,P_TYPE_TEMP,S_CONSTELLATION
0,OGLE-2016-BLG-1227L b,Microlensing,OGLE,2020-04-02,Mass,OGLE-2016-BLG-1227L,NaN,17h42m23.31s,-33d45m35.2s,Jovian,NaN,17 42 23.31,-33 45 35.20,NaN,Scorpius
1,Kepler-276 c,Transit,Kepler,2018-09-25,Mass,Kepler-276,NaN,19h34m16.37s,+39d02m10.72s,Neptunian,G,19 34 16.37,+39 02 10.72,Hot,Cygnus
2,Kepler-829 b,Transit,Kepler,2019-04-16,M-R relationship,Kepler-829,NaN,18h49m19.88s,+42d27m49.73s,Superterran,G,18 49 19.88,+42 27 49.73,Hot,Lyra
3,K2-283 b,Transit,K2,2019-09-05,M-R relationship,K2-283,NaN,00h52m46.65s,+09d41m34.51s,Neptunian,K,00 52 46.65,+09 41 34.51,Hot,Pisces
4,Kepler-477 b,Transit,Kepler,2019-04-16,M-R relationship,Kepler-477,NaN,19h12m16.19s,+42d21m19.10s,Superterran,K,19 12 16.19,+42 21 19.10,Hot,Lyra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5594,HD 222155 b,Radial Velocity,Haute-Provence Observatory,2023-06-12,Mass,HD 222155,G2 V,23h38m00.61s,+48d59m45.67s,Jovian,G,23 38 00.61,+48 59 45.67,Cold,Andromeda
5595,HD 88986 b,Radial Velocity,Haute-Provence Observatory,2023-12-04,Mass,HD 88986,G2 V,10h16m28.00s,+28d40m55.45s,Neptunian,G,10 16 28.00,+28 40 55.45,Hot,Leo Minor
5596,Kepler-30 b,Transit,Kepler,2023-11-28,Mass,Kepler-30,NaN,19h01m08.08s,+38d56m50.15s,Neptunian,G,19 01 08.08,+38 56 50.15,Hot,Lyra
5597,HD 3167 d,Radial Velocity,Multiple Observatories,2023-04-17,Msini,HD 3167,K0 V,00h34m57.64s,+04d22m50.60s,Superterran,K,00 34 57.64,+04 22 50.60,Hot,Pisces


We will consider only that planets which are likely to have rocky composition in an optimistic sample

In [ ]:
data_opt = data[(((data['P_MASS']<10.0) & (data['P_MASS']>0.1)) | ((data['P_RADIUS']<2.5) & (data['P_RADIUS']>0.5)))]
data_opt.head()

,P_NAME,P_DETECTION,P_DISCOVERY_FACILITY,P_YEAR,P_UPDATE,P_MASS,P_MASS_ORIGIN,P_RADIUS,P_PERIOD,P_SEMI_MAJOR_AXIS,P_ECCENTRICITY,P_INCLINATION,P_OMEGA,S_NAME,S_TYPE,S_RA,S_DEC,S_RA_STR,S_DEC_STR,S_MAG,S_DISTANCE,S_TEMPERATURE,S_MASS,S_RADIUS,S_METALLICITY,S_AGE,S_LOG_LUM,S_LOG_G,P_ESCAPE,P_POTENTIAL,P_GRAVITY,P_DENSITY,P_HILL_SPHERE,P_DISTANCE,P_PERIASTRON,P_APASTRON,P_DISTANCE_EFF,P_FLUX,P_FLUX_MIN,P_FLUX_MAX,P_TEMP_EQUIL,P_TEMP_EQUIL_MIN,P_TEMP_EQUIL_MAX,P_TEMP_SURF,P_TEMP_SURF_MIN,P_TEMP_SURF_MAX,P_TYPE,S_TYPE_TEMP,S_RA_TXT,S_DEC_TXT,S_LUMINOSITY,S_HZ_OPT_MIN,S_HZ_OPT_MAX,S_HZ_CON_MIN,S_HZ_CON_MAX,S_HZ_CON0_MIN,S_HZ_CON0_MAX,S_HZ_CON1_MIN,S_HZ_CON1_MAX,S_SNOW_LINE,S_ABIO_ZONE,S_TIDAL_LOCK,P_HABZONE_OPT,P_HABZONE_CON,P_TYPE_TEMP,P_HABITABLE,P_ESI,S_CONSTELLATION
2,Kepler-829 b,Transit,Kepler,2016,2019-04-16,5.100,M-R relationship,2.110,6.883376,0.06780,0.0000,89.820,NaN,Kepler-829,NaN,282.332830,42.463813,18h49m19.88s,+42d27m49.73s,14.973,1073.7600,5698.0,0.98,0.97,0.030,4.68,0.040,4.46,1.554690,2.417062,1.145527,0.542904,0.001175,0.067800,0.067800,0.067800,0.067800,238.528680,238.528680,238.528680,1001.11710,1001.11710,1001.11710,1130.60440,1130.60440,1130.60440,Superterran,G,18 49 19.88,+42 27 49.73,1.096478,0.789603,1.864342,1.000132,1.767566,1.057660,1.767566,0.965447,1.767566,2.827247,1.756317,0.459559,0,0,Hot,0,0.254763,Lyra
4,Kepler-477 b,Transit,Kepler,2016,2019-04-16,4.940,M-R relationship,2.070,11.119907,0.09110,0.0000,87.750,NaN,Kepler-477,NaN,288.067450,42.355305,19h12m16.19s,+42d21m19.10s,13.835,367.0000,5240.0,0.87,0.79,-0.040,2.09,-0.372,4.58,1.544822,2.386473,1.152886,0.556950,0.001626,0.091100,0.091100,0.091100,0.091100,51.163853,51.163853,51.163853,681.30334,681.30334,681.30334,769.42507,769.42507,769.42507,Superterran,K,19 12 16.19,+42 21 19.10,0.424620,0.504264,1.208168,0.638721,1.145452,0.675769,1.145452,0.616604,1.145452,1.759397,0.768502,0.386150,0,0,Hot,0,0.276721,Lyra
12,HD 28109 c,Transit,Transiting Exoplanet Survey Satellite (TESS),2022,2023-12-04,7.943,Mass,4.230,56.008190,0.30800,0.1203,89.543,NaN,HD 28109,F8/G0 V,65.238306,-68.102688,04h20m57.19s,-68d06m09.68s,9.420,139.6110,6120.0,1.26,1.45,0.000,1.10,0.456,4.13,1.370320,1.877778,0.443919,0.104945,0.005007,0.310229,0.270948,0.345052,0.308560,30.343390,24.001031,38.925068,596.25222,563.84169,636.29287,NaN,636.77059,718.59281,Neptunian,F,04 20 57.19,-68 06 09.68,2.857591,1.242505,2.903713,1.573772,2.752984,1.663692,2.752984,1.519093,2.752984,4.564190,3.943176,0.392567,0,0,Hot,0,0.206945,Hydrus
13,TOI-238 b,Transit,Transiting Exoplanet Survey Satellite (TESS),2024,2024-02-16,3.400,Mass,1.402,1.273099,0.02118,0.1800,84.600,NaN,TOI-238,K2V,349.231080,-18.606646,23h16m55.46s,-18d36m23.93s,10.748,80.5407,5059.0,0.79,0.73,-0.114,4.30,-0.471,4.58,1.557276,2.425107,1.729748,1.233772,0.000283,0.021523,0.017368,0.024992,0.021267,766.125920,541.232760,1120.780000,1331.98580,1228.69960,1473.94000,1504.26870,1387.62320,1664.58360,Terran,K,23 16 55.46,-18 36 23.93,0.338065,0.454118,1.095405,0.575206,1.038543,0.608690,1.038543,0.555296,1.038543,1.569870,0.545298,0.421708,0,0,Hot,0,0.284877,Aquarius
22,TOI-1470 b,Transit,Transiting Exoplanet Survey Satellite (TESS),2023,2023-08-07,7.320,Mass,2.180,2.527093,0.02850,0.3000,87.420,NaN,TOI-1470,M1.5 V,10.089143,61.213387,00h40m21.39s,+61d12m48.19s,13.459,51.9503,3709.0,0.47,0.47,0.000,1.30,-1.425,4.77,1.832430,3.357798,1.540274,0.706548,0.000498,0.029782,0.019950,0.037050,0.028831,48.505343,27.379378,94.430915,660.57708,582.71420,794.10560,746.01801,658.08413,896.81749,Superterran,M,00 40 21.39,+61 12 48.19,0.037584,0.158100,0.406026,0.200257,0.384944,0.212257,0.384944,0.193309,0.384944,0.523436,0.045102,0.290568,0,0,Hot,0,0.272496,Cassiopeia


In [ ]:
data_opt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3098 entries, 2 to 5597
Data columns (total 68 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   P_NAME                3098 non-null   object 
 1   P_DETECTION           3098 non-null   object 
 2   P_DISCOVERY_FACILITY  3098 non-null   object 
 3   P_YEAR                3098 non-null   int64  
 4   P_UPDATE              3098 non-null   object 
 5   P_MASS                3098 non-null   float64
 6   P_MASS_ORIGIN         3098 non-null   object 
 7   P_RADIUS              3098 non-null   float64
 8   P_PERIOD              3067 non-null   float64
 9   P_SEMI_MAJOR_AXIS     3098 non-null   float64
 10  P_ECCENTRICITY        2660 non-null   float64
 11  P_INCLINATION         2756 non-null   float64
 12  P_OMEGA               356 non-null    float64
 13  S_NAME                3098 non-null   object 
 14  S_TYPE                595 non-null    object 
 15  S_RA                  3098

In [ ]:
px.histogram(data_opt, x='P_MASS', title='Mass histogram')

In [ ]:
data_opt.sort_values(by='P_MASS', ascending=False)[['P_DENSITY', 'P_MASS', 'P_GRAVITY']]

,P_DENSITY,P_MASS,P_GRAVITY
2746,122.370400,1303.0000,269.214880
4977,49.763681,749.9000,122.916290
838,49.632366,603.8770,114.154440
1813,36.783854,508.5000,88.281250
904,36.571217,444.9620,84.113800
...,...,...,...
997,0.701970,0.1300,0.400123
2624,0.614359,0.1000,0.335440
3071,0.663547,0.0933,0.345044
5548,0.655856,0.0870,0.334487


In [ ]:
px.scatter(data_opt, x='P_ECCENTRICITY', y='S_MASS', color='P_TYPE')

Remove outliers if they're not habitable, don't know if i should do it

In [ ]:
# numeric_cols = data_opt.select_dtypes(include=['float64', 'int64']).columns.to_list()
# numeric_cols.remove('P_ESI')
# numeric_cols.remove('P_HABITABLE')
# numeric_cols.remove('P_OMEGA')
# for col in numeric_cols:
#   IQR = data_opt[col].quantile(0.99) - data_opt[col].quantile(0.01)
#   lower_bound = data_opt[col].quantile(0.01) - (1.5 * IQR)
#   upper_bound = data_opt[col].quantile(0.99) + (1.5 * IQR)
#   data_opt = data_opt[(data_opt[col] >= lower_bound) & (data_opt[col] <= upper_bound)]
#   print(f'{col} outliers removed')

In [ ]:
planets_per_star = data_opt.groupby('S_NAME').agg({'P_NAME': 'count'}).rename(columns={'P_NAME': 'PLANET_COUNT'})
planets_per_star['S_MASS'] = data_opt.groupby('S_NAME')['S_MASS'].mean()

In [ ]:
px.scatter(planets_per_star, x='PLANET_COUNT', y='S_MASS')

In [ ]:
px.scatter(data_opt, x='S_MASS', y='S_HZ_CON_MIN')

In [ ]:
px.scatter(data_opt, x='S_MASS', y='S_HZ_CON_MAX')

In [ ]:
corr_matrix = data_opt.corr(numeric_only=True)
px.imshow(corr_matrix)

In [ ]:
px.histogram(data_opt, x='P_HABITABLE', color='P_TYPE')

There is a big disproportion between classes, so up- and downsampling is required

In [ ]:
not_classification_columns = ['P_NAME', 'P_DETECTION', 'P_DISCOVERY_FACILITY', 'P_YEAR', 'P_UPDATE', 'P_OMEGA', 'S_TYPE', 'S_RA_STR', 'S_DEC_STR', 'S_RA_TXT', 'S_DEC_TXT']
classification_df = data_opt.drop(columns=not_classification_columns)
classification_df.head()

,P_MASS,P_MASS_ORIGIN,P_RADIUS,P_PERIOD,P_SEMI_MAJOR_AXIS,P_ECCENTRICITY,P_INCLINATION,S_NAME,S_RA,S_DEC,S_MAG,S_DISTANCE,S_TEMPERATURE,S_MASS,S_RADIUS,S_METALLICITY,S_AGE,S_LOG_LUM,S_LOG_G,P_ESCAPE,P_POTENTIAL,P_GRAVITY,P_DENSITY,P_HILL_SPHERE,P_DISTANCE,P_PERIASTRON,P_APASTRON,P_DISTANCE_EFF,P_FLUX,P_FLUX_MIN,P_FLUX_MAX,P_TEMP_EQUIL,P_TEMP_EQUIL_MIN,P_TEMP_EQUIL_MAX,P_TEMP_SURF,P_TEMP_SURF_MIN,P_TEMP_SURF_MAX,P_TYPE,S_TYPE_TEMP,S_LUMINOSITY,S_HZ_OPT_MIN,S_HZ_OPT_MAX,S_HZ_CON_MIN,S_HZ_CON_MAX,S_HZ_CON0_MIN,S_HZ_CON0_MAX,S_HZ_CON1_MIN,S_HZ_CON1_MAX,S_SNOW_LINE,S_ABIO_ZONE,S_TIDAL_LOCK,P_HABZONE_OPT,P_HABZONE_CON,P_TYPE_TEMP,P_HABITABLE,P_ESI,S_CONSTELLATION
2,5.100,M-R relationship,2.110,6.883376,0.06780,0.0000,89.820,Kepler-829,282.332830,42.463813,14.973,1073.7600,5698.0,0.98,0.97,0.030,4.68,0.040,4.46,1.554690,2.417062,1.145527,0.542904,0.001175,0.067800,0.067800,0.067800,0.067800,238.528680,238.528680,238.528680,1001.11710,1001.11710,1001.11710,1130.60440,1130.60440,1130.60440,Superterran,G,1.096478,0.789603,1.864342,1.000132,1.767566,1.057660,1.767566,0.965447,1.767566,2.827247,1.756317,0.459559,0,0,Hot,0,0.254763,Lyra
4,4.940,M-R relationship,2.070,11.119907,0.09110,0.0000,87.750,Kepler-477,288.067450,42.355305,13.835,367.0000,5240.0,0.87,0.79,-0.040,2.09,-0.372,4.58,1.544822,2.386473,1.152886,0.556950,0.001626,0.091100,0.091100,0.091100,0.091100,51.163853,51.163853,51.163853,681.30334,681.30334,681.30334,769.42507,769.42507,769.42507,Superterran,K,0.424620,0.504264,1.208168,0.638721,1.145452,0.675769,1.145452,0.616604,1.145452,1.759397,0.768502,0.386150,0,0,Hot,0,0.276721,Lyra
12,7.943,Mass,4.230,56.008190,0.30800,0.1203,89.543,HD 28109,65.238306,-68.102688,9.420,139.6110,6120.0,1.26,1.45,0.000,1.10,0.456,4.13,1.370320,1.877778,0.443919,0.104945,0.005007,0.310229,0.270948,0.345052,0.308560,30.343390,24.001031,38.925068,596.25222,563.84169,636.29287,NaN,636.77059,718.59281,Neptunian,F,2.857591,1.242505,2.903713,1.573772,2.752984,1.663692,2.752984,1.519093,2.752984,4.564190,3.943176,0.392567,0,0,Hot,0,0.206945,Hydrus
13,3.400,Mass,1.402,1.273099,0.02118,0.1800,84.600,TOI-238,349.231080,-18.606646,10.748,80.5407,5059.0,0.79,0.73,-0.114,4.30,-0.471,4.58,1.557276,2.425107,1.729748,1.233772,0.000283,0.021523,0.017368,0.024992,0.021267,766.125920,541.232760,1120.780000,1331.98580,1228.69960,1473.94000,1504.26870,1387.62320,1664.58360,Terran,K,0.338065,0.454118,1.095405,0.575206,1.038543,0.608690,1.038543,0.555296,1.038543,1.569870,0.545298,0.421708,0,0,Hot,0,0.284877,Aquarius
22,7.320,Mass,2.180,2.527093,0.02850,0.3000,87.420,TOI-1470,10.089143,61.213387,13.459,51.9503,3709.0,0.47,0.47,0.000,1.30,-1.425,4.77,1.832430,3.357798,1.540274,0.706548,0.000498,0.029782,0.019950,0.037050,0.028831,48.505343,27.379378,94.430915,660.57708,582.71420,794.10560,746.01801,658.08413,896.81749,Superterran,M,0.037584,0.158100,0.406026,0.200257,0.384944,0.212257,0.384944,0.193309,0.384944,0.523436,0.045102,0.290568,0,0,Hot,0,0.272496,Cassiopeia


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

scaler = StandardScaler()
encoder = LabelEncoder()
p_esi = classification_df['P_ESI']
classification_df.drop(columns='P_ESI', inplace=True)
classification_df.drop(columns='P_HABITABLE', inplace=True)
numeric_cols = classification_df.select_dtypes(include=['float64', 'int64']).columns.to_list()
categorical_cols = classification_df.select_dtypes(include='object').columns.to_list()

classification_df[numeric_cols] = scaler.fit_transform(classification_df[numeric_cols])
classification_df[categorical_cols] = classification_df[categorical_cols].apply(encoder.fit_transform)
classification_df['P_ESI'] = p_esi
classification_df.head()


,P_MASS,P_MASS_ORIGIN,P_RADIUS,P_PERIOD,P_SEMI_MAJOR_AXIS,P_ECCENTRICITY,P_INCLINATION,S_NAME,S_RA,S_DEC,S_MAG,S_DISTANCE,S_TEMPERATURE,S_MASS,S_RADIUS,S_METALLICITY,S_AGE,S_LOG_LUM,S_LOG_G,P_ESCAPE,P_POTENTIAL,P_GRAVITY,P_DENSITY,P_HILL_SPHERE,P_DISTANCE,P_PERIASTRON,P_APASTRON,P_DISTANCE_EFF,P_FLUX,P_FLUX_MIN,P_FLUX_MAX,P_TEMP_EQUIL,P_TEMP_EQUIL_MIN,P_TEMP_EQUIL_MAX,P_TEMP_SURF,P_TEMP_SURF_MIN,P_TEMP_SURF_MAX,P_TYPE,S_TYPE_TEMP,S_LUMINOSITY,S_HZ_OPT_MIN,S_HZ_OPT_MAX,S_HZ_CON_MIN,S_HZ_CON_MAX,S_HZ_CON0_MIN,S_HZ_CON0_MAX,S_HZ_CON1_MIN,S_HZ_CON1_MAX,S_SNOW_LINE,S_ABIO_ZONE,S_TIDAL_LOCK,P_HABZONE_OPT,P_HABZONE_CON,P_TYPE_TEMP,S_CONSTELLATION,P_ESI
2,-0.052408,0,0.242144,-0.087606,-0.230545,-0.324549,0.549067,1949,0.311966,0.422628,0.527685,0.650097,0.429885,0.463846,0.217394,0.304680,0.085540,0.533904,-0.225006,-0.022879,-0.062621,-0.070184,-0.053858,-0.245153,-0.229546,-0.239269,-0.220016,-0.230321,-0.030515,-0.030095,-0.031340,0.422503,0.435920,0.399153,0.325015,0.435919,0.399152,4,3,0.046359,0.347452,0.347299,0.347459,0.347300,0.347544,0.347300,0.347508,0.347300,0.333540,-0.025561,0.622565,-0.165919,-0.12545,1,41,0.254763
4,-0.057274,0,0.185741,-0.065906,-0.151875,-0.324549,-0.027953,1683,0.388361,0.418786,0.000220,-0.424324,-0.023702,0.019456,-0.348685,-0.121941,-0.753923,-0.079273,0.365448,-0.033217,-0.064576,-0.069564,-0.053070,-0.163290,-0.151805,-0.153096,-0.148684,-0.151889,-0.044336,-0.043916,-0.045159,-0.356199,-0.342423,-0.375959,-0.416004,-0.342423,-0.375959,4,4,-0.467709,-0.404949,-0.401798,-0.404940,-0.401797,-0.404786,-0.401797,-0.404882,-0.401797,-0.420267,-0.025561,-0.480956,-0.165919,-0.12545,1,41,0.276721
12,0.034051,1,3.231499,0.164016,0.580473,1.306276,0.471853,128,-2.580113,-3.491862,-2.046138,-0.770003,0.847818,1.595019,1.726937,0.121843,-1.074798,1.153035,-1.848753,-0.216013,-0.097091,-0.129302,-0.078427,0.451138,0.579331,0.512056,0.628776,0.580119,-0.045872,-0.045920,-0.046062,-0.563287,-0.628294,-0.485048,NaN,-0.628294,-0.485048,2,2,1.393864,1.541696,1.533859,1.541686,1.533858,1.541434,1.533858,1.541625,1.533858,1.559667,-0.025561,-0.384488,-0.165919,-0.12545,1,33,0.206945
13,-0.104107,1,-0.756187,-0.116342,-0.387954,2.115588,-0.906026,2190,1.203167,-1.739507,-1.430608,-0.859802,-0.202958,-0.303736,-0.537378,-0.572940,-0.037625,-0.226614,0.365448,-0.020171,-0.062107,-0.020957,-0.015101,-0.407382,-0.383952,-0.425789,-0.351069,-0.386960,0.008403,-0.007765,0.033731,1.228123,0.989796,1.545102,1.091649,0.989796,1.545102,5,4,-0.533936,-0.537178,-0.530530,-0.537168,-0.530529,-0.536932,-0.530529,-0.537112,-0.530529,-0.554055,-0.025561,0.053575,-0.165919,-0.12545,1,2,0.284877
22,0.015105,1,0.340849,-0.109919,-0.363239,3.742345,-0.119941,2160,-3.314796,1.086436,-0.174056,-0.903265,-1.539948,-1.596506,-1.355047,0.121843,-1.009975,-1.646448,1.300333,0.268063,-0.002490,-0.036923,-0.044678,-0.368170,-0.356394,-0.416238,-0.314155,-0.361499,-0.044532,-0.045671,-0.041968,-0.406665,-0.582364,-0.102567,-0.464027,-0.582364,-0.102567,4,5,-0.763848,-1.317738,-1.317534,-1.317753,-1.317537,-1.317910,-1.317537,-1.317853,-1.317537,-1.292744,-0.025561,-1.917781,-0.165919,-0.12545,1,15,0.272496


In [ ]:
classification_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3098 entries, 2 to 5597
Data columns (total 56 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   P_MASS             3098 non-null   float64
 1   P_MASS_ORIGIN      3098 non-null   int64  
 2   P_RADIUS           3098 non-null   float64
 3   P_PERIOD           3067 non-null   float64
 4   P_SEMI_MAJOR_AXIS  3098 non-null   float64
 5   P_ECCENTRICITY     2660 non-null   float64
 6   P_INCLINATION      2756 non-null   float64
 7   S_NAME             3098 non-null   int64  
 8   S_RA               3098 non-null   float64
 9   S_DEC              3098 non-null   float64
 10  S_MAG              3063 non-null   float64
 11  S_DISTANCE         3089 non-null   float64
 12  S_TEMPERATURE      3063 non-null   float64
 13  S_MASS             3098 non-null   float64
 14  S_RADIUS           3063 non-null   float64
 15  S_METALLICITY      2977 non-null   float64
 16  S_AGE              2466 non-n

In [ ]:
classification_df.drop(columns='S_AGE', inplace=True)
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
classification_df = pd.DataFrame(imputer.fit_transform(classification_df), columns=classification_df.columns)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

tsne = TSNE(n_components = 2, random_state=42)
tsne_df = tsne.fit_transform(classification_df)

tsne_df = pd.DataFrame(tsne_df, columns=['tsne1', 'tsne2'])
tsne_df['P_ESI'] = classification_df['P_ESI']
tsne_df.head()

,tsne1,tsne2,P_ESI
0,65.408607,-17.082918,0.254763
1,36.573570,-12.193113,0.276721
2,-70.241409,4.784081,0.206945
3,52.914360,39.589115,0.284877
4,55.978634,36.521317,0.272496


In [ ]:
px.scatter(tsne_df, x='tsne1', y='tsne2', color='P_ESI')

In [ ]:
from sklearn.model_selection import train_test_split

X = classification_df.drop(columns='P_ESI')
y = classification_df['P_ESI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import torch

training_data = torch.utils.data.TensorDataset(torch.tensor(X_train.values, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32))
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test.values, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.float32))



In [ ]:
classification_df.shape

(3098, 55)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class PlanetSimilarityRegressor(nn.Module):
  def __init__(self, input_size):
    super(PlanetSimilarityRegressor, self).__init__()
    self.fc1 = nn.Linear(input_size, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 1)
    self.sigmoid = nn.Sigmoid()
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = self.sigmoid(x)
    return x.squeeze()

def training_loop(model: PlanetSimilarityRegressor, train: torch.utils.data.dataloader):
  model.train()
  loss = 0
  for idx, (data, target) in enumerate(train):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
  return loss

def validation_loop(model: PlanetSimilarityRegressor, val: torch.utils.data.dataloader):
  model.eval()
  test_loss = 0
  with torch.no_grad():
    for idx, (data, target) in enumerate(val):
      output = model(data)
      loss = criterion(output, target)
      test_loss += loss.item()
      return test_loss





In [ ]:
model = PlanetSimilarityRegressor(input_size=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 100
train_loader = torch.utils.data.DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)
for epoch in range(EPOCHS):
  train_loss = training_loop(model, train_loader)
  test_loss = validation_loop(model, test_loader)
  print(f'Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss} | Test Loss: {test_loss}')

Epoch 1/100 | Train Loss: 0.010681423358619213 | Test Loss: 0.01629025489091873
Epoch 2/100 | Train Loss: 0.004807099234312773 | Test Loss: 0.004955184645950794
Epoch 3/100 | Train Loss: 0.009898190386593342 | Test Loss: 0.007990493439137936
Epoch 4/100 | Train Loss: 0.00677183922380209 | Test Loss: 0.007642942015081644
Epoch 5/100 | Train Loss: 0.005015665199607611 | Test Loss: 0.0030641169287264347
Epoch 6/100 | Train Loss: 0.001994306920096278 | Test Loss: 0.0032792689744383097
Epoch 7/100 | Train Loss: 0.009870404377579689 | Test Loss: 0.00838883314281702
Epoch 8/100 | Train Loss: 0.0029832185246050358 | Test Loss: 0.009992286562919617
Epoch 9/100 | Train Loss: 0.0019177069189026952 | Test Loss: 0.0023453915491700172
Epoch 10/100 | Train Loss: 0.005823198240250349 | Test Loss: 0.032882798463106155
Epoch 11/100 | Train Loss: 0.002380554797127843 | Test Loss: 0.003924699500203133
Epoch 12/100 | Train Loss: 0.0007639934192411602 | Test Loss: 0.004312257282435894
Epoch 13/100 | Train L